In [2]:
import matplotlib

matplotlib.use('TkAgg')
import LDP.basicDP.OLHbasic as lhb
import LDP.basicFunction.basicfunc as bf
import LDP.basicDP.LDPKVbasic as lkvb
import LDP.basicDP.MLPKVbasic as mlkvp
import numpy as np
import random
import time


"""数据处理"""
# 关闭科学计数法显示
np.set_printoptions(suppress=True)

##### 读取文件
##### 构建kv对元组列表，每一行代表一个用户的kv对list

In [3]:
k_path='../LDPdataset/Clothing/data/Clothing_k.txt'
v_path='../LDPdataset/Clothing/data/Clothing_v.txt'
lable_path='../LDPdataset/Clothing/data/Clothing_lable.txt'

data_k = bf.readtxt(k_path)
data_v = bf.readtxt(v_path)
data_l = bf.readtxt(lable_path)

n = len(data_k)  # 用户数量
d = len(data_l[0])  # key域的长度
epsilon = 5

# 构建kv对元组列表，每一行代表一个用户的kv对list
kv = []

for i in range(n):
    data_k[i]=list(map(int,data_k[i]))
    tmp = zip(data_k[i], data_v[i])
    kv.append(list(tmp))

# 构建label标签，我已经把标签规范化为[1,l]的数字了，也就是说不需要用之前的原始标签了,不用查字典了
label=[i+1 for i in range(d)]
kv

[[(213, 1.0), (92, 1.0)],
 [(4007, 1.0)],
 [(3029, 1.0), (2492, 1.0)],
 [(2125, 1.0)],
 [(20, 0.6), (48, 1.0), (1244, 1.0), (2696, 0.6), (58, 0.6)],
 [(1973, 1.0)],
 [(2261, 1.0)],
 [(3867, 1.0)],
 [(481, 1.0)],
 [(2084, 1.0)],
 [(1707, 0.6)],
 [(30, 1.0)],
 [(3601, 1.0)],
 [(57, 0.6), (11, 1.0)],
 [(2625, 0.6)],
 [(613, 1.0), (3181, 1.0)],
 [(2021, 1.0)],
 [(1475, 1.0), (1777, 0.6)],
 [(651, 1.0)],
 [(3815, 0.6)],
 [(98, 0.2), (12, 1.0), (690, 1.0)],
 [(70, 0.6), (3867, 0.6)],
 [(550, 1.0)],
 [(683, 1.0), (822, 1.0), (11, 1.0), (28, 1.0), (103, 1.0)],
 [(3, 1.0)],
 [(1178, 1.0)],
 [(892, 1.0), (2750, 1.0), (854, 1.0), (2309, 1.0)],
 [(1034, 1.0)],
 [(1037, 0.6)],
 [(99, 1.0)],
 [(1698, 1.0)],
 [(25, 1.0)],
 [(63, 1.0)],
 [(2521, 1.0)],
 [(1958, 1.0), (902, 1.0)],
 [(1475, 1.0), (1271, 1.0)],
 [(3297, 1.0), (3452, 0.6)],
 [(8, 0.6)],
 [(2163, 1.0)],
 [(1, 1.0)],
 [(3328, 1.0), (1826, 0.6)],
 [(1029, 0.6)],
 [(75, -0.2)],
 [(1538, 1.0)],
 [(16, 1.0)],
 [(1367, 1.0)],
 [(1607, 0.6)],
 [(

##### 分组，GroupA完成候选项集，GroupB完成填充项集，GroupC完成估计

In [4]:

GroupA,GroupB,GroupC=mlkvp.divide_list(kv,1,1,1)

n_A=len(GroupA) #A组用户数量
n_B=len(GroupB) #B组用户数量
n_C=len(GroupC) #C组用户数量


##### GroupA用户完成Step1和2
##### Step1：采样并扰动上传

In [5]:
GroupA

[[(213, 1.0), (92, 1.0)],
 [(4007, 1.0)],
 [(3029, 1.0), (2492, 1.0)],
 [(2125, 1.0)],
 [(20, 0.6), (48, 1.0), (1244, 1.0), (2696, 0.6), (58, 0.6)],
 [(1973, 1.0)],
 [(2261, 1.0)],
 [(3867, 1.0)],
 [(481, 1.0)],
 [(2084, 1.0)],
 [(1707, 0.6)],
 [(30, 1.0)],
 [(3601, 1.0)],
 [(57, 0.6), (11, 1.0)],
 [(2625, 0.6)],
 [(613, 1.0), (3181, 1.0)],
 [(2021, 1.0)],
 [(1475, 1.0), (1777, 0.6)],
 [(651, 1.0)],
 [(3815, 0.6)],
 [(98, 0.2), (12, 1.0), (690, 1.0)],
 [(70, 0.6), (3867, 0.6)],
 [(550, 1.0)],
 [(683, 1.0), (822, 1.0), (11, 1.0), (28, 1.0), (103, 1.0)],
 [(3, 1.0)],
 [(1178, 1.0)],
 [(892, 1.0), (2750, 1.0), (854, 1.0), (2309, 1.0)],
 [(1034, 1.0)],
 [(1037, 0.6)],
 [(99, 1.0)],
 [(1698, 1.0)],
 [(25, 1.0)],
 [(63, 1.0)],
 [(2521, 1.0)],
 [(1958, 1.0), (902, 1.0)],
 [(1475, 1.0), (1271, 1.0)],
 [(3297, 1.0), (3452, 0.6)],
 [(8, 0.6)],
 [(2163, 1.0)],
 [(1, 1.0)],
 [(3328, 1.0), (1826, 0.6)],
 [(1029, 0.6)],
 [(75, -0.2)],
 [(1538, 1.0)],
 [(16, 1.0)],
 [(1367, 1.0)],
 [(1607, 0.6)],
 [(

In [6]:
#从每个用户项集里随机采样一个key并扰动
sampling_perturbed_A=[] 
for i in range(n_A):
    random.seed(1)
    sampling_key=random.sample(GroupA[i],1)[0][0] #随机采样得到的项
    perturbed_key=lhb.OLH_perturbation(epsilon,sampling_key,i)
    sampling_perturbed_A.append((perturbed_key,i))
sampling_perturbed_A

[(123, 0),
 (123, 1),
 (34, 2),
 (83, 3),
 (47, 4),
 (46, 5),
 (64, 6),
 (73, 7),
 (114, 8),
 (37, 9),
 (94, 10),
 (118, 11),
 (74, 12),
 (81, 13),
 (99, 14),
 (120, 15),
 (104, 16),
 (52, 17),
 (85, 18),
 (115, 19),
 (24, 20),
 (46, 21),
 (140, 22),
 (129, 23),
 (13, 24),
 (51, 25),
 (118, 26),
 (54, 27),
 (118, 28),
 (68, 29),
 (28, 30),
 (95, 31),
 (78, 32),
 (109, 33),
 (110, 34),
 (121, 35),
 (59, 36),
 (24, 37),
 (41, 38),
 (99, 39),
 (29, 40),
 (36, 41),
 (15, 42),
 (93, 43),
 (17, 44),
 (42, 45),
 (22, 46),
 (27, 47),
 (44, 48),
 (140, 49),
 (120, 50),
 (122, 51),
 (10, 52),
 (112, 53),
 (95, 54),
 (2, 55),
 (8, 56),
 (109, 57),
 (33, 58),
 (119, 59),
 (106, 60),
 (1, 61),
 (30, 62),
 (139, 63),
 (82, 64),
 (96, 65),
 (34, 66),
 (135, 67),
 (52, 68),
 (62, 69),
 (23, 70),
 (7, 71),
 (141, 72),
 (93, 73),
 (36, 74),
 (25, 75),
 (144, 76),
 (79, 77),
 (13, 78),
 (91, 79),
 (93, 80),
 (118, 81),
 (0, 82),
 (128, 83),
 (79, 84),
 (128, 85),
 (146, 86),
 (140, 87),
 (126, 88),
 (41,

##### Step2：聚合得到候选集

In [7]:
#Step1的估计结果

starttime1 = time.clock()
print("开始执行OLH...")
# 注意这里的用户数目n要用组A的用户数目，而不是所有的
est_A=lhb.OLH_aggregation_mutilprocess(epsilon,sampling_perturbed_A,n_A,d,lhb.get_support,8)
# 程序运行结束时间
endtime1 = time.clock()
print('运行时间:', endtime1 - starttime1)
est_A

开始执行OLH...
ForkPoolWorker-1 启动
ForkPoolWorker-3 启动
ForkPoolWorker-2 启动
ForkPoolWorker-4 启动
ForkPoolWorker-5 启动
ForkPoolWorker-6 启动
ForkPoolWorker-7 启动
ForkPoolWorker-8 启动
[40, 57, 41, 33, 33, 40, 27, 57, 31, 32, 48, 41, 30, 31, 60, 38, 43, 42, 32, 52, 53, 30, 43, 35, 22, 40, 38, 56, 35, 38, 34, 36, 33, 35, 32, 34, 33, 50, 27, 36, 35, 30, 25, 40, 35, 30, 25, 70, 33, 45, 25, 30, 32, 33, 28, 32, 34, 31, 24, 40, 37, 33, 31, 43, 24, 50, 34, 38, 27, 43, 30, 35, 33, 23, 43, 30, 32, 43, 28, 32, 40, 30, 22, 29, 29, 38, 25, 39, 45, 38, 28, 33, 36, 35, 33, 37, 29, 43, 37, 29, 52, 37, 31, 31, 27, 22, 29, 27, 32, 21, 28, 23, 32, 38, 28, 38, 33, 45, 23, 22, 35, 34, 28, 37, 34, 36, 31, 33, 25, 27, 19, 27, 33, 38, 28, 28, 38, 28, 38, 32, 20, 35, 27, 31, 26, 35, 36, 28, 32, 34, 30, 31, 34, 41, 38, 30, 30, 33, 33, 39, 20, 38, 33, 28, 34, 40, 35, 28, 23, 26, 40, 37, 35, 36, 29, 32, 30, 28, 32, 36, 30, 24, 28, 30, 33, 33, 21, 41, 42, 31, 29, 29, 24, 29, 22, 22, 24, 35, 28, 29, 35, 23, 29, 32, 30, 30, 37, 

[147.42436803282283,
 459.17439557693416,
 212.2035945354953,
 121.10780726611213,
 94.79124649940144,
 183.86268294057612,
 181.8383321123676,
 532.0510253924407,
 78.59643987373332,
 147.42436803282283,
 355.9325033382999,
 90.74254484298442,
 151.47306968923988,
 25.963318340311922,
 214.22794536370384,
 76.5720890455248,
 242.56885695862306,
 297.22632932025294,
 82.64514153015035,
 145.40001720461433,
 493.58835965647893,
 27.98766916852044,
 70.49903656089926,
 52.27987910702262,
 40.13377413777153,
 9.768511714643802,
 104.91300064044401,
 339.73769671263176,
 32.03637082493747,
 319.4941884305466,
 159.57047300207392,
 113.01040395327807,
 52.27987910702262,
 123.13215809432066,
 19.890265855686376,
 60.377282419856684,
 30.012019996728956,
 143.3756663764058,
 66.45033490448223,
 139.32696471998878,
 96.81559732760995,
 70.49903656089926,
 88.71819401477589,
 92.76689567119293,
 106.93735146865254,
 3.695459230018256,
 -28.69415402131799,
 503.7101137975215,
 143.3756663764058

In [8]:
#用做对比时间，这是没有用多线程的，上面是用了多进程的EC-OLH

# starttime2 = time.clock()
# print("开始执行OLH...")
# support=[]
# g = int(np.exp(epsilon)) + 1  # 参数g
# for i in range(d):
#     c=0
#     for j in range(len(sampling_perturbed_A)):
#         if(lhb.olh_hash(i+1,g,sampling_perturbed_A[j][1])==sampling_perturbed_A[j][0]):
#             c+=1
#     support.append(c)
# 
# #support
# 
# e=[]
# p = lhb.p_values(epsilon, n=g)  # 参数p
# 
# for i in range(len(support)):
#     es=lhb.aggregation(support[i],len(GroupA),g,p)
#     e.append(es)
# endtime2 = time.clock()
# print('运行时间:', endtime2 - starttime2)
# e

In [9]:
k=20
est_A_temp=list(zip(label,est_A))
est_A_temp.sort(key=lambda x:x[1],reverse=True)

 #候选集
candi_dict=dict(est_A_temp[:2*k])
Candidate=list(candi_dict.keys())

Candidate

[8,
 48,
 21,
 2,
 101,
 11,
 28,
 30,
 18,
 17,
 89,
 15,
 3,
 67,
 6,
 7,
 98,
 64,
 31,
 13,
 88,
 1,
 10,
 20,
 38,
 49,
 1973,
 40,
 3462,
 66,
 34,
 92,
 4,
 155,
 4380,
 3261,
 32,
 133,
 93,
 1411]

##### GroupB用户完成Step3和4
##### Step3：取交集并扰动上传

In [10]:
GroupB

[[(966, 0.6)],
 [(1033, 1.0), (2955, 1.0)],
 [(48, 0.6)],
 [(3253, 1.0), (689, 1.0)],
 [(31, 1.0), (2006, 1.0)],
 [(2753, 1.0)],
 [(89, 0.6)],
 [(481, 1.0)],
 [(1918, 0.6)],
 [(43, 0.6), (5628, 0.6), (1682, 1.0)],
 [(8, 1.0)],
 [(2309, 1.0)],
 [(3759, 1.0)],
 [(1087, 1.0)],
 [(718, 0.6)],
 [(3783, 1.0)],
 [(5, 0.6)],
 [(1721, 1.0)],
 [(1078, 1.0),
  (3232, 0.6),
  (4355, 1.0),
  (2626, 1.0),
  (1292, 1.0),
  (1127, 0.2),
  (1153, 0.6),
  (512, 0.6),
  (3719, 1.0),
  (5095, 0.2),
  (3247, 1.0),
  (1497, 1.0),
  (3303, 1.0),
  (3004, 1.0),
  (220, 1.0),
  (1355, 1.0),
  (3367, 1.0),
  (1165, 1.0),
  (3065, 1.0),
  (2205, 1.0),
  (5153, 0.6),
  (1336, 1.0),
  (701, 0.6),
  (390, 1.0),
  (5486, 1.0),
  (3666, 0.6),
  (768, 0.6),
  (4641, 1.0),
  (4003, 0.6),
  (702, 0.6),
  (1992, 1.0),
  (3472, 0.6),
  (504, 0.6),
  (5118, 1.0),
  (237, 1.0)],
 [(58, 1.0), (3822, 1.0), (15, 1.0), (69, 1.0)],
 [(187, 1.0)],
 [(1973, 1.0)],
 [(31, 1.0)],
 [(66, 0.6), (93, 1.0), (2521, -0.2), (101, 0.6)],
 [

In [11]:
GroupB_k=[] #GroupB组用户拥有的k值
GroupB_num=[] #交集的个数
num_perturbed_B=[] #交集个数的扰动
# 获取GroupB用户拥有的k值
for i in range(n_B):
    each_k=list(zip(*GroupB[i]))[0]
    GroupB_k.append(each_k)
    num=len(set(Candidate).intersection(set(each_k)))
    GroupB_num.append(num)
    perturbed_num=lhb.OLH_perturbation(epsilon,num,i)
    num_perturbed_B.append((perturbed_num,i))
GroupB_k

[(966,),
 (1033, 2955),
 (48,),
 (3253, 689),
 (31, 2006),
 (2753,),
 (89,),
 (481,),
 (1918,),
 (43, 5628, 1682),
 (8,),
 (2309,),
 (3759,),
 (1087,),
 (718,),
 (3783,),
 (5,),
 (1721,),
 (1078,
  3232,
  4355,
  2626,
  1292,
  1127,
  1153,
  512,
  3719,
  5095,
  3247,
  1497,
  3303,
  3004,
  220,
  1355,
  3367,
  1165,
  3065,
  2205,
  5153,
  1336,
  701,
  390,
  5486,
  3666,
  768,
  4641,
  4003,
  702,
  1992,
  3472,
  504,
  5118,
  237),
 (58, 3822, 15, 69),
 (187,),
 (1973,),
 (31,),
 (66, 93, 2521, 101),
 (2786,),
 (5,),
 (2565,),
 (13,),
 (4289,
  2775,
  93,
  1813,
  3371,
  3769,
  1974,
  3642,
  5680,
  912,
  2692,
  4425,
  2220,
  307),
 (2718,),
 (18, 2460, 746),
 (21,),
 (2054, 2476),
 (3348, 1333, 300),
 (1702, 835),
 (5088,),
 (2525, 6, 2958),
 (64,),
 (101,),
 (1870,),
 (3094, 31, 3211, 1252),
 (2595,),
 (2,),
 (2538, 3234, 2176),
 (1857,),
 (2,),
 (2984, 2428, 2030, 3744, 261, 3914, 3403, 3739),
 (2847,),
 (916,),
 (8,),
 (61,),
 (30,),
 (1660,),
 (2

In [12]:
num_perturbed_B

[(24, 0),
 (22, 1),
 (127, 2),
 (42, 3),
 (63, 4),
 (15, 5),
 (55, 6),
 (55, 7),
 (12, 8),
 (14, 9),
 (139, 10),
 (127, 11),
 (24, 12),
 (42, 13),
 (75, 14),
 (37, 15),
 (0, 16),
 (62, 17),
 (53, 18),
 (95, 19),
 (87, 20),
 (117, 21),
 (138, 22),
 (30, 23),
 (22, 24),
 (61, 25),
 (142, 26),
 (43, 27),
 (25, 28),
 (86, 29),
 (45, 30),
 (109, 31),
 (142, 32),
 (12, 33),
 (143, 34),
 (82, 35),
 (105, 36),
 (41, 37),
 (115, 38),
 (40, 39),
 (103, 40),
 (48, 41),
 (28, 42),
 (111, 43),
 (43, 44),
 (57, 45),
 (26, 46),
 (131, 47),
 (8, 48),
 (51, 49),
 (51, 50),
 (141, 51),
 (12, 52),
 (92, 53),
 (100, 54),
 (106, 55),
 (128, 56),
 (60, 57),
 (38, 58),
 (43, 59),
 (36, 60),
 (74, 61),
 (35, 62),
 (43, 63),
 (20, 64),
 (93, 65),
 (135, 66),
 (78, 67),
 (30, 68),
 (144, 69),
 (119, 70),
 (17, 71),
 (138, 72),
 (11, 73),
 (59, 74),
 (16, 75),
 (108, 76),
 (38, 77),
 (29, 78),
 (8, 79),
 (51, 80),
 (92, 81),
 (59, 82),
 (3, 83),
 (77, 84),
 (108, 85),
 (88, 86),
 (129, 87),
 (83, 88),
 (123, 89)

In [13]:
starttime3 = time.clock()
print("开始执行OLH...")
# 注意这里的用户数目n要用组B的用户数目，而不是所有的,并且数据维度也不是d了，而是[0,2k]，因为交集的数目最多是候选集的长度，即2k
est_B=lhb.OLH_aggregation_mutilprocess(epsilon,num_perturbed_B,n_B,2*k,lhb.get_support,8)
# 程序运行结束时间
endtime3 = time.clock()
print('运行时间:', endtime3 - starttime3)
est_B# 项数从0开始



开始执行OLH...
ForkPoolWorker-9 启动
ForkPoolWorker-11 启动
ForkPoolWorker-10 启动
ForkPoolWorker-13 启动
ForkPoolWorker-12 启动
ForkPoolWorker-14 启动
ForkPoolWorker-15 启动
ForkPoolWorker-16 启动
[533, 57, 28, 31, 26, 42, 26, 34, 21, 30, 23, 33, 22, 30, 26, 32, 27, 27, 35, 26, 26, 23, 39, 32, 26, 29, 33, 28, 33, 23, 41, 35, 40, 38, 20, 39, 24, 38, 20, 31]
[553, 75, 32, 31, 21, 28, 30, 31, 41, 37, 30, 26, 32, 25, 31, 29, 32, 26, 30, 38, 34, 33, 28, 29, 26, 27, 39, 33, 25, 29, 30, 30, 32, 40, 35, 33, 26, 31, 36, 20]
[527, 54, 24, 34, 25, 21, 30, 26, 20, 31, 28, 25, 29, 25, 31, 31, 36, 36, 30, 32, 27, 37, 31, 27, 27, 26, 27, 24, 37, 30, 33, 29, 30, 21, 31, 30, 36, 41, 32, 39]
[570, 58, 36, 23, 26, 33, 26, 40, 32, 24, 23, 30, 24, 32, 22, 37, 33, 26, 28, 37, 22, 26, 27, 31, 23, 28, 29, 30, 27, 34, 34, 27, 32, 35, 31, 40, 27, 33, 27, 29]
[550, 87, 37, 30, 32, 31, 31, 31, 26, 27, 34, 26, 31, 36, 29, 26, 33, 32, 26, 35, 24, 39, 23, 49, 37, 24, 29, 34, 22, 32, 28, 21, 26, 32, 26, 26, 26, 31, 26, 31]
[524, 67, 29

[8333.899117308058,
 578.6110944412366,
 72.52338738910777,
 -18.572399880275412,
 -63.10811810086275,
 32.03637082493747,
 -46.913311475194625,
 76.5720890455248,
 3.695459230018256,
 -0.3532424263987743,
 -18.572399880275412,
 19.890265855686376,
 9.768511714643802,
 -18.572399880275412,
 -4.401944082815804,
 7.744160886435287,
 19.890265855686376,
 -20.596750708483928,
 -14.52369822385838,
 44.18247579418856,
 -61.08376727265423,
 25.963318340311922,
 15.841564199269348,
 34.06072165314598,
 9.768511714643802,
 7.744160886435287,
 -8.450645739232835,
 -8.450645739232835,
 -18.572399880275412,
 -28.69415402131799,
 50.255528278814104,
 -42.864609818777595,
 5.719810058226772,
 36.0850724813545,
 -6.42629491102432,
 32.03637082493747,
 3.695459230018256,
 58.352931591648165,
 3.695459230018256,
 -2.3775932546072895]

##### Step4：获得填充长度返回

In [14]:
#得到填充项长度
padding_length=2

##### GroupC用户完成Step5和6
##### Step5：填充采样并上传

##### (1)GroupC也和Candidate取交集

In [15]:
GroupC_inter=[] #GroupC组用户和Candidate的交集
GroupC_k=[] #GroupC组用户拥有的k值
GroupC_v=[] #GroupC组用户拥有的v值
for i in range(n_C):
    each_k=list(zip(*GroupC[i]))[0]
    temp_k=set(Candidate).intersection(set(each_k))
    GroupC_dict=dict(GroupC[i])
    temp_v=list(map(GroupC_dict.get,temp_k))
    GroupC_inter.append(list(zip(temp_k,temp_v)))
GroupC_inter

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [(64, 1.0)],
 [],
 [],
 [],
 [],
 [],
 [(11, 1.0)],
 [(48, 1.0), (11, 0.6)],
 [(67, 1.0)],
 [],
 [],
 [],
 [(1, 0.6)],
 [],
 [],
 [(17, 1.0)],
 [],
 [(7, 1.0)],
 [(20, 1.0)],
 [(17, 1.0)],
 [(31, 0.6)],
 [],
 [(49, 0.2)],
 [(155, 1.0)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [(101, 0.6)],
 [],
 [(93, 1.0)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [(49, 0.2)],
 [],
 [],
 [],
 [(3, 1.0)],
 [],
 [(28, 1.0)],
 [],
 [(10, 0.2), (3, 0.2), (4, 0.6)],
 [],
 [],
 [],
 [],
 [(30, 0.6)],
 [(2, 1.0)],
 [(30, 1.0)],
 [],
 [],
 [],
 [],
 [],
 [],
 [(20, 1.0)],
 [],
 [(8, 1.0)],
 [],
 [],
 [],
 [],
 [],
 [],
 [(2, 0.6), (11, 1.0)],
 [],
 [],
 [],
 [],
 [(28, 1.0)],
 [],
 [],
 [],
 [],
 [(92, 1.0)],
 [],
 [],
 [(6, 0.6)],
 [],
 [],
 [(8, 1.0)],
 [(17, 0.6)],
 [(2, 1.0)],
 [],
 [(17, 1.0)],
 [],
 [(21, 1.0), (6, 1.0)],
 [],
 [],
 [],
 [],
 [],
 [(8, 0.6)],
 [],
 [],
 [(1, 1.0

##### (2)Padding and Sampling

In [16]:
pad_item=[i+1 for i in range(d,d+padding_length)]
pad_label=Candidate+pad_item
PS_C=[mlkvp.P_S(GroupC_inter[i],d,padding_length) for i in range(n_C)]
PS_C

[(5851, 1),
 (5852, 1),
 (5851, -1),
 (5852, 1),
 (5851, -1),
 (5852, 1),
 (5851, 1),
 (5852, -1),
 (5851, -1),
 (5851, -1),
 (5851, 1),
 (5851, 1),
 (5852, 1),
 (5851, 1),
 (5851, -1),
 (5851, -1),
 (5852, 1),
 (5852, -1),
 (5852, -1),
 (5852, -1),
 (5851, -1),
 (11, 1),
 (11, 1),
 (67, 1),
 (5851, 1),
 (5851, 1),
 (5851, 1),
 (1, 1),
 (5852, 1),
 (5851, 1),
 (17, 1),
 (5852, 1),
 (7, 1),
 (5852, -1),
 (17, 1),
 (31, 1),
 (5851, -1),
 (5852, -1),
 (155, 1),
 (5851, -1),
 (5852, -1),
 (5852, 1),
 (5851, -1),
 (5851, 1),
 (5852, -1),
 (5851, -1),
 (5852, 1),
 (5851, 1),
 (5851, 1),
 (5852, -1),
 (5851, -1),
 (5852, -1),
 (5851, 1),
 (101, 1),
 (5852, -1),
 (5851, 1),
 (5851, 1),
 (5852, 1),
 (5851, 1),
 (5852, 1),
 (5852, -1),
 (5851, 1),
 (5852, 1),
 (5852, 1),
 (5851, 1),
 (5851, 1),
 (5852, 1),
 (5851, -1),
 (5851, 1),
 (5851, -1),
 (5852, -1),
 (5851, 1),
 (5851, -1),
 (5852, -1),
 (28, 1),
 (5851, -1),
 (3, -1),
 (5852, 1),
 (5852, 1),
 (5852, -1),
 (5852, -1),
 (5852, 1),
 (5851, 

##### (3)UE扰动

In [17]:
eps1=np.log((np.exp(epsilon)+1)/2)
eps2=epsilon
a=0.5
p=lkvb.p_values(eps2)
b=1-lkvb.p_values(eps1)
perturbed_C=mlkvp.PCKV_UE(PS_C,2*k,padding_length,pad_label,a,p,b)
perturbed_C

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

##### Step6：估计频率和均值

In [18]:
f_k,m_k=mlkvp.AEC_UE(perturbed_C,2*k,padding_length,n_C,a,p,b)
print(f_k)
print(m_k)

[0.019166318220994905, 0.016364325567077113, 0.018349070363602206, 0.020450564854040554, 0.015430328015771188, 0.012628335361853395, 0.0099430924018488603, 0.011927837198373951, 0.016831324342730075, 0.010293341483588582, 0.0074913488296707905, 0.013212083831419599, 0.0084253463809767157, 0.0060903525027118941, 0.010410091177501822, 0.0076080985235840309, 0.0087755954627164456, 0.0084253463809767157, 0.0032883598487941099, 0.003171610154880869, 2.8416356454775368e-05, 0.004572606481839765, 0.0023543622974881843, 0.0027046113792279065, 0.003171610154880869, 0.003171610154880869, 0.0077248482174972714, 0.0044558567879265246, 0.001537114440095492, 0.0025878616853146656, 0.0057401034209721719, 0.0058568531148854132, 0.0086588457688032052, 0.0058568531148854132, 0.004572606481839765, 0.0020041132157484621, 0.0043391070940132833, 0.0032883598487941099, 0.0079583476053237522, 0.0036386089305338321]
[0.9316954656265537, 0.78145836763800869, 0.79111106205861426, 0.57461352242442898, 0.873557871

In [25]:
est_f=[] #估计频率的排名
for i in range(2*k):
    est_f.append((Candidate[i],f_k[i]))
est_f.sort(key=lambda x:x[1],reverse=True)
est_k=dict(est_f) # 估计频率字典
est_m=dict(zip(Candidate,m_k)) #估计均值字典
print('est_k:',est_k)
print('est_m:',est_m)

est_k: {2: 0.020450564854040554, 8: 0.019166318220994905, 21: 0.018349070363602206, 18: 0.016831324342730075, 48: 0.016364325567077113, 101: 0.015430328015771188, 15: 0.013212083831419599, 11: 0.012628335361853395, 30: 0.011927837198373951, 6: 0.010410091177501822, 17: 0.010293341483588582, 28: 0.0099430924018488603, 98: 0.0087755954627164456, 4: 0.0086588457688032052, 3: 0.0084253463809767157, 64: 0.0084253463809767157, 93: 0.0079583476053237522, 1973: 0.0077248482174972714, 7: 0.0076080985235840309, 89: 0.0074913488296707905, 67: 0.0060903525027118941, 92: 0.0058568531148854132, 155: 0.0058568531148854132, 34: 0.0057401034209721719, 1: 0.004572606481839765, 4380: 0.004572606481839765, 40: 0.0044558567879265246, 32: 0.0043391070940132833, 1411: 0.0036386089305338321, 31: 0.0032883598487941099, 133: 0.0032883598487941099, 13: 0.003171610154880869, 38: 0.003171610154880869, 49: 0.003171610154880869, 20: 0.0027046113792279065, 66: 0.0025878616853146656, 10: 0.0023543622974881843, 3261: 0

##### 获取实际结果

In [26]:
true_path='../LDPdataset/Clothing/data/Clothing_id_count_avg.txt'
true_count= bf.readtxt(true_path)
true_k={} #真实频率字典
true_v={} #真实均值字典
true_data=[] # 真实频率排名元组list
for i in range(d):
    true_k[i+1]=true_count[i][1]/n
    true_v[i+1]=true_count[i][2]*0.2-1 #归一化到[-1,1]
    true_data.append((i+1,true_count[i][1]))

true_data.sort(key=lambda x:x[1],reverse=True)
topk_dict=dict(true_data)
true_rank=list(topk_dict.keys())#真实频率排名list
print("true_rank:",true_rank)
print("true_mean:",true_v)

true_rank: [8, 101, 2, 21, 48, 11, 28, 30, 18, 98, 17, 15, 6, 89, 3, 64, 29, 1, 67, 7, 2285, 1973, 93, 92, 32, 66, 13, 42, 49, 4, 31, 63, 52, 5, 10, 1731, 34, 23, 53, 16, 1270, 40, 20, 3601, 83, 88, 38, 41, 24, 1314, 27, 19, 3620, 44, 142, 2309, 12, 2600, 3261, 1073, 116, 3328, 3236, 25, 57, 2261, 99, 1647, 125, 3257, 154, 1474, 39, 1949, 683, 3717, 9, 854, 68, 1932, 722, 86, 54, 37, 2040, 80, 254, 3799, 3773, 2687, 651, 1873, 3212, 3360, 3411, 3121, 36, 3842, 120, 1084, 3183, 3410, 1411, 2497, 3417, 3859, 1525, 2995, 122, 3330, 85, 2907, 3645, 14, 133, 22, 2292, 2890, 1848, 1690, 3356, 110, 4195, 108, 2853, 2521, 2958, 4816, 61, 2824, 796, 1328, 3600, 1224, 2993, 3098, 3477, 43, 1573, 5561, 2253, 95, 2825, 2742, 1914, 3348, 75, 3462, 118, 3211, 300, 1540, 3789, 71, 153, 3184, 3656, 69, 281, 724, 46, 2116, 3585, 2428, 1244, 2313, 2889, 35, 2456, 3258, 90, 145, 1175, 1504, 2782, 26, 2352, 2922, 165, 2861, 1053, 148, 2892, 70, 966, 3249, 5409, 2181, 2750, 45, 3783, 883, 81, 102, 689, 138

In [27]:
true_data
true_rank

[8,
 101,
 2,
 21,
 48,
 11,
 28,
 30,
 18,
 98,
 17,
 15,
 6,
 89,
 3,
 64,
 29,
 1,
 67,
 7,
 2285,
 1973,
 93,
 92,
 32,
 66,
 13,
 42,
 49,
 4,
 31,
 63,
 52,
 5,
 10,
 1731,
 34,
 23,
 53,
 16,
 1270,
 40,
 20,
 3601,
 83,
 88,
 38,
 41,
 24,
 1314,
 27,
 19,
 3620,
 44,
 142,
 2309,
 12,
 2600,
 3261,
 1073,
 116,
 3328,
 3236,
 25,
 57,
 2261,
 99,
 1647,
 125,
 3257,
 154,
 1474,
 39,
 1949,
 683,
 3717,
 9,
 854,
 68,
 1932,
 722,
 86,
 54,
 37,
 2040,
 80,
 254,
 3799,
 3773,
 2687,
 651,
 1873,
 3212,
 3360,
 3411,
 3121,
 36,
 3842,
 120,
 1084,
 3183,
 3410,
 1411,
 2497,
 3417,
 3859,
 1525,
 2995,
 122,
 3330,
 85,
 2907,
 3645,
 14,
 133,
 22,
 2292,
 2890,
 1848,
 1690,
 3356,
 110,
 4195,
 108,
 2853,
 2521,
 2958,
 4816,
 61,
 2824,
 796,
 1328,
 3600,
 1224,
 2993,
 3098,
 3477,
 43,
 1573,
 5561,
 2253,
 95,
 2825,
 2742,
 1914,
 3348,
 75,
 3462,
 118,
 3211,
 300,
 1540,
 3789,
 71,
 153,
 3184,
 3656,
 69,
 281,
 724,
 46,
 2116,
 3585,
 2428,
 1244,
 2313,
 288

##### 指标

##### 1.命中率

In [28]:
hit_ratio=mlkvp.hit_ratio(Candidate,true_rank,k)
hit_ratio

0.9

##### 2.MSE_f

In [29]:
MSE_f=mlkvp.MSE(Candidate,est_k,true_k,k)
MSE_f

6.1139416449530551e-06

##### 3.MSE_m

In [30]:
MSE_m=mlkvp.MSE(Candidate,est_m,true_v,k)
MSE_m

0.11801563475521945